In [1]:
import itertools
class Jeu:
    def __init__(self,joueurs,coalitions,structure_coalition,imputation,caracteristique ):
        self.joueurs = joueurs
        self.coalitions = coalitions
        self.structure_coalition = structure_coalition
        self.imputation = imputation
        self.caracteristique = caracteristique

    def isMonotone(self):
        for i in self.coalitions:
          val1 = self.caracteristique[i] # u(C)
          for item in i : 
            val2 = self.caracteristique.get(frozenset(item)) # u(C') tel que C' inclus dans C
            if val1 < val2 : # u(C) < u(C')
              return False
        return True
    
    def isSuperAdditive(self):
        for coalition1 in self.coalitions:
            for coalition2 in self.coalitions:
                if(coalition1 != coalition2): # C != C'
                    coalitionUnion = coalition1.union(coalition2) # (C U C')
                    if(coalitionUnion in self.coalitions and coalitionUnion != coalition1 and coalitionUnion != coalition2): # (C U C') in C et (C U C') != C et (C U C') != C'
                        if(self.caracteristique[coalitionUnion] < self.caracteristique[coalition1] + self.caracteristique[coalition2]): # v(C U C') < v(C) + v(C')
                            return False
        return True


    def isConvexe(self):
        for coalition1 in self.coalitions:
            for coalition2 in self.coalitions:
                coalitionUnion = coalition1.union(coalition2) # (C U C')
                coalitionIntersection = coalition1.intersection(coalition2) # (C n C')
                if coalitionUnion in self.coalitions and coalitionIntersection in self.coalitions: # (C U C') in C et (C n C') in C
                    # v(C U C') + v(C n C') < v(C) + v(C')
                    if(self.caracteristique[coalitionUnion] + self.caracteristique[coalitionIntersection] < self.caracteristique[coalition1] + self.caracteristique[coalition2]):
                        return False
        return True
    
    def isVeto(self):
        listVeto = []
        for joueur in self.joueurs:
            is_Veto = True
            for coalition in self.coalitions:
                if not joueur in coalition:
                    if(self.caracteristique[coalition] != 0): # Si il existe une coalition sans le joueur qui n'a pas une valeur nulle
                        is_Veto = False #Le joueur n'est pas un veto
            if is_Veto:
                listVeto.append(joueur)
        return listVeto
                    
    def valeur_Shapley(self,joueur):
        somme = 0
        permutations = list(itertools.permutations(self.joueurs)) # Liste des permutations des joueurs
        for permutation in permutations:
            joueurs_avant = set() # Ensemble des joueurs avant le joueur concerné
            while len(joueurs_avant) < len(permutation) and permutation[len(joueurs_avant)] != joueur:
                joueurs_avant.add(permutation[len(joueurs_avant)])  
            somme += self.caracteristique[frozenset(joueurs_avant)] # Somme des valeurs des joueurs avant le joueur concerné
        
        return somme/len(permutations) # Somme divisée par le nombre de permutations
            


In [2]:
joueurs = ["A","B","C"] 
coalition_set = set()
caracteristique = {}

for i in range(1, len(joueurs) + 1):
    for subset in itertools.combinations(joueurs, i):
        coalition_set.add(frozenset(subset))
        structure_coalition = set()

# Q.1 Test monotonicité d'un jeu 

In [3]:
caracteristique[frozenset("")] = 0
caracteristique[frozenset({"A"})] = 0
caracteristique[frozenset({"B"})] = 0
caracteristique[frozenset({"C"})] = 0
caracteristique[frozenset({"A","B"})] = 500
caracteristique[frozenset({"A","C"})] = 500
caracteristique[frozenset({"B","C"})] = 750
caracteristique[frozenset({"A","B","C"})] = 1000


for coalition in coalition_set:
    for coalition2 in coalition_set:
        if not coalition.intersection(coalition2):
            structure_coalition.add((coalition,coalition2))

imputation = {"A": 4, "B": 5, "C": 3}

jeu = Jeu(joueurs,coalition_set,structure_coalition,imputation,caracteristique)
print("Le jeu est monotone ?",jeu.isMonotone())

caracteristique[frozenset({"A"})] = 700

print("Le jeu est monotone ?",jeu.isMonotone())

Le jeu est monotone ? True
Le jeu est monotone ? False


# Q.2 Test superadditivité

In [4]:
caracteristique[frozenset("")] = 0
caracteristique[frozenset({"A"})] = 0
caracteristique[frozenset({"B"})] = 0
caracteristique[frozenset({"C"})] = 0
caracteristique[frozenset({"A","B"})] = 1
caracteristique[frozenset({"A","C"})] = 1
caracteristique[frozenset({"B","C"})] = 1
caracteristique[frozenset({"A","B","C"})] = 10


for coalition in coalition_set:
    for coalition2 in coalition_set:
        if not coalition.intersection(coalition2):
            structure_coalition.add((coalition,coalition2))

imputation = {"A": 4, "B": 5, "C": 3}

jeu = Jeu(joueurs,coalition_set,structure_coalition,imputation,caracteristique)
print("Le jeu est superadditive ?",jeu.isSuperAdditive())

caracteristique[frozenset({"A"})] = 700

print("Le jeu est superadditive ?",jeu.isSuperAdditive())

Le jeu est superadditive ? True
Le jeu est superadditive ? False


# Q.3 Test Convexité

In [5]:
caracteristique[frozenset("")] = 0
caracteristique[frozenset({"A"})] =0
caracteristique[frozenset({"B"})] = 0
caracteristique[frozenset({"C"})] = 0
caracteristique[frozenset({"A","B"})] = 1000
caracteristique[frozenset({"A","C"})] = 1000
caracteristique[frozenset({"B","C"})] = 1000
caracteristique[frozenset({"A","B","C"})] = 1250


for coalition in coalition_set:
    for coalition2 in coalition_set:
        if not coalition.intersection(coalition2):
            structure_coalition.add((coalition,coalition2))

imputation = {"A": 4, "B": 5, "C": 3}

jeu = Jeu(joueurs,coalition_set,structure_coalition,imputation,caracteristique)
print("Le jeu est convexe ?",jeu.isConvexe())

caracteristique[frozenset({"A","B","C"})] = 3000

print("Le jeu est convexe ?",jeu.isConvexe())

Le jeu est convexe ? False
Le jeu est convexe ? True


# Q.4 Le cœur d’un jeu simple

In [6]:
caracteristique[frozenset("")] = 0
caracteristique[frozenset({"A"})] = 0
caracteristique[frozenset({"B"})] = 0
caracteristique[frozenset({"C"})] = 0
caracteristique[frozenset({"A","B"})] = 0
caracteristique[frozenset({"A","C"})] = 0
caracteristique[frozenset({"B","C"})] = 1
caracteristique[frozenset({"A","B","C"})] = 1

print("Le coeur du jeu simple est:",jeu.isVeto())

caracteristique[frozenset({"A","B"})] = 1
caracteristique[frozenset({"A","C"})] = 1

print("Le coeur du jeu simple est:",jeu.isVeto())

Le coeur du jeu simple est: ['B', 'C']
Le coeur du jeu simple est: []


# Q.5 La valeur de Shapley

In [7]:
caracteristique[frozenset("")] = 0
caracteristique[frozenset({"A"})] = 0
caracteristique[frozenset({"B"})] = 0
caracteristique[frozenset({"C"})] = 0
caracteristique[frozenset({"A","B"})] = 500
caracteristique[frozenset({"A","C"})] = 500
caracteristique[frozenset({"B","C"})] = 750
caracteristique[frozenset({"A","B","C"})] = 1000

print("La valeur de Shapley de A est :",jeu.valeur_Shapley("A"))

La valeur de Shapley de A est : 250.0
